In [1]:
# !pip install folktables

In [2]:
#info about Data
# https://arxiv.org/pdf/2108.04884.pdf

#ACSDataSource ,ACSEmployment, ACSPublicCoverage, ACSIncome

-> (using ACSEmployment) from the paper: predict whether an individual is employed, after filtering the ACS PUMS data
sample to only include individuals between the ages of 16 and 90  

In [1]:
# !pip install torchsummary

In [2]:
# !python --version

In [1]:
import pandas as pd
import numpy as np
import copy

import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# detail about the state: load_acs.py

In [3]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
data_source = ACSDataSource(survey_year='2017', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [4]:
# features_np, label_np, group_np = ACSEmployment.df_to_numpy(acs_data)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [5]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0


In [6]:
label

,ESR
0,False
1,True
2,True
3,False
4,False
...,...
47640,False
47641,True
47642,False
47643,False


In [7]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [8]:
len(merged_df)

47645

In [9]:
group.value_counts()

RAC1P
1        34827
2        10641
9          821
6          617
8          468
3          207
5           47
7           15
4            2
Name: count, dtype: int64

In [10]:
merged_df["RAC1P"].value_counts()

# 1: White alone – 2: Black or African American alone

RAC1P
1.0    34827
2.0    10641
9.0      821
6.0      617
8.0      468
3.0      207
5.0       47
7.0       15
4.0        2
Name: count, dtype: int64

In [11]:
pd.unique(merged_df["RAC1P"])

array([2., 1., 9., 6., 3., 8., 5., 7., 4.])

In [12]:
#  for Black and white people
filtered_df= merged_df[["SEX","RAC1P","ESR"]][merged_df['RAC1P'].isin([1,2])]
print("len(filtered_df): ", len(filtered_df),"\n")
print(filtered_df["ESR"].value_counts(),"\n")
print(filtered_df["SEX"].value_counts(),"\n")
print(filtered_df["RAC1P"].value_counts(),"\n")
filtered_df.head() 

len(filtered_df):  45468 

ESR
False    27041
True     18427
Name: count, dtype: int64 

SEX
2.0    23697
1.0    21771
Name: count, dtype: int64 

RAC1P
1.0    34827
2.0    10641
Name: count, dtype: int64 



,SEX,RAC1P,ESR
0,1.0,2.0,False
1,2.0,1.0,True
2,1.0,1.0,True
3,2.0,1.0,False
4,1.0,1.0,False


In [13]:
# 1- Male 2- Female
# 1- White 2-Black

counts_frq = filtered_df.groupby(['SEX', 'RAC1P']).size()
counts_frq

SEX  RAC1P
1.0  1.0      16783
     2.0       4988
2.0  1.0      18044
     2.0       5653
dtype: int64

In [14]:
merged_df

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,73.0,10.0,4.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
1,31.0,21.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,True
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
3,48.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47640,20.0,18.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47641,18.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47642,25.0,17.0,1.0,16.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,9.0,False
47643,24.0,17.0,5.0,16.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False


In [15]:
# # filter the dataframe for each combination

#White Men
WM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 1)]
#Black Men
BM = merged_df.loc[(merged_df['SEX'] == 1) & (merged_df['RAC1P'] == 2)]
#white Women
WW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 1)]
#Black Women
BW = merged_df.loc[(merged_df['SEX'] == 2) & (merged_df['RAC1P'] == 2)]

# Removing Sex and Gen from 4 Groups

In [16]:
WM.columns

Index(['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC',
       'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR'],
      dtype='object')

In [17]:
WM_updated=WM
BM_updated=BM
WW_updated=WW
BW_updated=BW

# Grouping

In [18]:
#Group-1
#Train set: White men+black men+Black women—- validation set: black men
WM_BM_BW_train = pd.concat([WM_updated,BM_updated,BW_updated])
BM_validation=BM_updated.copy()

#Group-2
#Train set: White women+black women+black men—- validation set: black women
WW_BW_BM_train=pd.concat([WW_updated,BW_updated,BM_updated])
BW_validation=BW_updated.copy()

#Group-3
#Train set: black men+white men+white women+validation set: white men
BM_WM_WW_train=pd.concat([BM_updated,WM_updated,WW_updated])
WM_validation=WM_updated.copy()

#Group-4
#Train set: white women+white men+black women, validation set: white women
WW_WM_BW_train=pd.concat([WW_updated,WM_updated,BW_updated])
WW_validation=WW_updated.copy()

In [19]:

print("BM" ,len(BM),"RAC: ", pd.unique(BM['RAC1P']),"SEX: ", pd.unique(BM['SEX']))
print("BW" ,len(BW),"RAC: ", pd.unique(BW['RAC1P']),"SEX: ", pd.unique(BW['SEX']))
print("WM" ,len(WM),"RAC: ", pd.unique(WM['RAC1P']),"SEX: ", pd.unique(WM['SEX']))
print("WW" ,len(WW),"RAC: ", pd.unique(WW['RAC1P']),"SEX: ", pd.unique(WW['SEX']))

BM 4988 RAC:  [2.] SEX:  [1.]
BW 5653 RAC:  [2.] SEX:  [2.]
WM 16783 RAC:  [1.] SEX:  [1.]
WW 18044 RAC:  [1.] SEX:  [2.]


In [21]:
data_to_use_train1= WM_BM_BW_train
data_to_use_val1= BM_validation

data_to_use_train2= WW_BW_BM_train
data_to_use_val2= BW_validation


data_to_use_train3= BM_WM_WW_train
data_to_use_val3= WM_validation


data_to_use_train4= WW_WM_BW_train
data_to_use_val4= WW_validation

In [22]:
data_to_use_train1.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,True
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
10,32.0,16.0,1.0,0.0,2.0,0.0,5.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,True
18,80.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,False
19,32.0,22.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True


In [23]:
X_train1= data_to_use_train1[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_train1_sens= data_to_use_train1[['SEX']]
y_train1= data_to_use_train1[['ESR']]


X_train2= data_to_use_train2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_train2_sens= data_to_use_train2[['SEX']]
y_train2= data_to_use_train2[['ESR']]

X_train3= data_to_use_train3[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_train3_sens= data_to_use_train3[['SEX']]
y_train3= data_to_use_train3[['ESR']]

X_train4= data_to_use_train4[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_train4_sens= data_to_use_train4[['SEX']]
y_train4= data_to_use_train4[['ESR']]



In [24]:
len(X_train1),len(X_train2),len(X_train3),len(X_train4)

(27424, 28685, 39815, 40480)

In [27]:
X_valid1 = data_to_use_val1[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_valid1_sens= data_to_use_val1[['SEX']]
y_valid1 = data_to_use_val1[['ESR']]

X_valid2 = data_to_use_val2[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_valid2_sens= data_to_use_val2[['SEX']]
y_valid2 = data_to_use_val2[['ESR']]

X_valid3 = data_to_use_val3[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_valid3_sens= data_to_use_val3[['SEX']]
y_valid3 = data_to_use_val3[['ESR']]

X_valid4 = data_to_use_val4[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM']]
X_valid4_sens= data_to_use_val4[['SEX']]
y_valid4 = data_to_use_val4[['ESR']]

# X_valid = torch.from_numpy(X_valid.to_numpy()).float()
# y_valid = torch.squeeze(torch.from_numpy(y_valid.to_numpy()).float())

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [28]:
print("TYPE X: ",type(X_train1)," Type Y", type(y_train1),"\n")
print("len(X_train),len(y_train), len(X_val),len(y_val): ",len(X_train1),len(y_train1), len(X_valid1),len(y_valid1))
print("X_train.shape, y_train.shape ",X_train1.shape, y_train1.shape,"\n")
print("X_valid.shape, y_valid.shape ",X_valid1.shape, y_valid1.shape,"\n")

X_train1

TYPE X:  <class 'pandas.core.frame.DataFrame'>  Type Y <class 'pandas.core.frame.DataFrame'> 

len(X_train),len(y_train), len(X_val),len(y_val):  27424 27424 4988 4988
X_train.shape, y_train.shape  (27424, 14) (27424, 1) 

X_valid.shape, y_valid.shape  (4988, 14) (4988, 1) 



,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM
2,41.0,17.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
4,16.0,13.0,5.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0
10,32.0,16.0,1.0,0.0,2.0,0.0,5.0,3.0,4.0,1.0,2.0,2.0,2.0,2.0
18,80.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
19,32.0,22.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47605,24.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0
47623,20.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0
47628,17.0,16.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0
47635,19.0,19.0,5.0,17.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0


In [29]:
X_valid1_sens

,SEX
0,1.0
7,1.0
13,1.0
15,1.0
16,1.0
...,...
47630,1.0
47631,1.0
47633,1.0
47636,1.0


In [30]:
y_valid4

,ESR
1,True
3,False
17,True
21,False
24,False
...,...
47620,False
47627,False
47634,False
47641,True


In [31]:
# scaler = StandardScaler()

In [32]:
# X = scaler.fit_transform(X)
# X = scaler.fit_transform(X)
# X = scaler.fit_transform(X)
# X = scaler.fit_transform(X)

In [33]:
X_valid1 = preprocessing.normalize(X_valid1)
X_train1 = preprocessing.normalize(X_train1)
X_train1_sens=X_train1_sens['SEX'].map({1: 1, 2: 0}).to_numpy()
X_valid1_sens=X_valid1_sens['SEX'].map({1: 1, 2: 0}).to_numpy()

X_valid2 = preprocessing.normalize(X_valid2)
X_train2 = preprocessing.normalize(X_train2)
X_train2_sens=X_train2_sens['SEX'].map({1: 1, 2: 0}).to_numpy()
X_valid2_sens=X_valid2_sens['SEX'].map({1: 1, 2: 0}).to_numpy()

X_valid3 = preprocessing.normalize(X_valid3)
X_train3 = preprocessing.normalize(X_train3)
X_train3_sens=X_train3_sens['SEX'].map({1: 1, 2: 0}).to_numpy()
X_valid3_sens=X_valid3_sens['SEX'].map({1: 1, 2: 0}).to_numpy()

X_valid4 = preprocessing.normalize(X_valid4)
X_train4 = preprocessing.normalize(X_train4)
X_train4_sens=X_train4_sens['SEX'].map({1: 1, 2: 0}).to_numpy()
X_valid4_sens=X_valid4_sens['SEX'].map({1: 1, 2: 0}).to_numpy()


In [34]:
encoder = LabelEncoder()

In [46]:
# Binary encoding of labels
op=encoder.fit(y_train1)

/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
op = encoder.transform(y_train1)

encoder.inverse_transform(op)

/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([ True, False,  True, ...,  True, False, False])

In [37]:
op

array([1, 0, 1, ..., 1, 0, 0])

In [38]:
y_train1 = encoder.transform(y_train1)
y_valid1 = encoder.transform(y_valid1)

y_train2 = encoder.transform(y_train2)
y_valid2 = encoder.transform(y_valid2)

y_train3 = encoder.transform(y_train3)
y_valid3 = encoder.transform(y_valid3)

y_train4 = encoder.transform(y_train4)
y_valid4 = encoder.transform(y_valid4)

/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/chiragapandav/Downloads/Hiwi/gpu_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/chiragapandav/Downl

In [39]:
# Convert to 2D PyTorch tensors
X_train1= torch.tensor(X_train1, dtype=torch.float32)
X_valid1= torch.tensor(X_valid1, dtype=torch.float32)
X_train1_sens = torch.tensor(X_train1_sens, dtype=torch.float32).reshape(-1, 1)

y_train1 = torch.tensor(y_train1, dtype=torch.float32).reshape(-1, 1)
y_valid1 = torch.tensor(y_valid1, dtype=torch.float32).reshape(-1, 1)
X_valid1_sens = torch.tensor(X_valid1_sens, dtype=torch.float32).reshape(-1, 1)


X_train2 = torch.tensor(X_train2, dtype=torch.float32)
X_valid2 = torch.tensor(X_valid2, dtype=torch.float32)
X_train2_sens = torch.tensor(X_train2_sens, dtype=torch.float32).reshape(-1, 1)
y_train2 = torch.tensor(y_train2, dtype=torch.float32).reshape(-1, 1)
y_valid2 = torch.tensor(y_valid2, dtype=torch.float32).reshape(-1, 1)
X_valid2_sens = torch.tensor(X_valid2_sens, dtype=torch.float32).reshape(-1, 1)


X_train3= torch.tensor(X_train3, dtype=torch.float32)
X_valid3= torch.tensor(X_valid3, dtype=torch.float32)
X_train3_sens = torch.tensor(X_train3_sens, dtype=torch.float32).reshape(-1, 1)
y_train3 = torch.tensor(y_train3, dtype=torch.float32).reshape(-1, 1)
y_valid3 = torch.tensor(y_valid3, dtype=torch.float32).reshape(-1, 1)
X_valid3_sens = torch.tensor(X_valid3_sens, dtype=torch.float32).reshape(-1, 1)



X_train4= torch.tensor(X_train4, dtype=torch.float32)
X_valid4= torch.tensor(X_valid4, dtype=torch.float32)
X_train4_sens = torch.tensor(X_train4_sens, dtype=torch.float32).reshape(-1, 1)
y_train4 = torch.tensor(y_train4, dtype=torch.float32).reshape(-1, 1)
y_valid4 = torch.tensor(y_valid4, dtype=torch.float32).reshape(-1, 1)
X_valid4_sens = torch.tensor(X_valid4_sens, dtype=torch.float32).reshape(-1, 1)

In [40]:
X_train1,y_train1

(tensor([[0.9152, 0.3795, 0.0223,  ..., 0.0446, 0.0446, 0.0446],
         [0.7318, 0.5946, 0.2287,  ..., 0.0915, 0.0915, 0.0915],
         [0.8703, 0.4351, 0.0272,  ..., 0.0544, 0.0544, 0.0544],
         ...,
         [0.5648, 0.5316, 0.1661,  ..., 0.0664, 0.0664, 0.0664],
         [0.5803, 0.5803, 0.1527,  ..., 0.0611, 0.0611, 0.0611],
         [0.6742, 0.5394, 0.1348,  ..., 0.0539, 0.0539, 0.0539]]),
 tensor([[1.],
         [0.],
         [1.],
         ...,
         [1.],
         [0.],
         [0.]]))

In [41]:
y_train1,y_valid1,X_train1_sens 

(tensor([[1.],
         [0.],
         [1.],
         ...,
         [1.],
         [0.],
         [0.]]),
 tensor([[0.],
         [0.],
         [1.],
         ...,
         [0.],
         [0.],
         [0.]]),
 tensor([[1.],
         [1.],
         [1.],
         ...,
         [0.],
         [0.],
         [0.]]))

In [42]:
X_train1.shape, y_train1.shape

(torch.Size([27424, 14]), torch.Size([27424, 1]))

# data loader-torch

In [43]:
from torch.utils.data import DataLoader, Subset, random_split
import torch.utils.data as data_utils
import pickle
# Passing to DataLoader
BATCH=64

train1 = data_utils.TensorDataset(X_train1, y_train1,X_train1_sens)
valid1 = data_utils.TensorDataset(X_valid1, y_valid1,X_valid1_sens)

train2 = data_utils.TensorDataset(X_train2, y_train2,X_train2_sens)
valid2 = data_utils.TensorDataset(X_valid2, y_valid2,X_valid2_sens)

train3 = data_utils.TensorDataset(X_train3, y_train3,X_train3_sens)
valid3 = data_utils.TensorDataset(X_valid3, y_valid3,X_valid3_sens)

train4 = data_utils.TensorDataset(X_train4, y_train4,X_train4_sens)
valid4 = data_utils.TensorDataset(X_valid4, y_valid4,X_valid4_sens)

trainloader1 = data_utils.DataLoader(train1, batch_size=BATCH, shuffle=True)
trainloader2 = data_utils.DataLoader(train2, batch_size=BATCH, shuffle=True)
trainloader3 = data_utils.DataLoader(train3, batch_size=BATCH, shuffle=True)
trainloader4 = data_utils.DataLoader(train4, batch_size=BATCH, shuffle=True)

valloader1 = data_utils.DataLoader(valid1, batch_size=BATCH)
valloader2 = data_utils.DataLoader(valid2, batch_size=BATCH)
valloader3 = data_utils.DataLoader(valid3, batch_size=BATCH)
valloader4 = data_utils.DataLoader(valid4, batch_size=BATCH)


In [44]:
train_data_all_client=[trainloader1,trainloader2,trainloader3,trainloader4]
val_data_all_client=[valloader1,valloader2,valloader3,valloader4]

In [45]:
with open("testing_client.pkl", "wb") as f:
    pickle.dump(train_data_all_client, f)
    
with open("valid_testing_client.pkl", "wb") as f:
    pickle.dump(val_data_all_client, f)